# Document-document Distance Matrix

## Import packages

In [1]:
import numpy as np
import pandas as pd
import string
import re
from gensim.utils import lemmatize, simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import collections
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from nltk.corpus import stopwords
stop_words = stopwords.words('portuguese')
stop_words.extend(['janeiro','fevereiro','março','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro','recife'])


In [2]:
qnt = 1000

## Import dataset

In [3]:
data = pd.read_csv("dore_2009_proc.csv")
data.head()

dtext = data["conteudo"]
dtext = dtext[:qnt]

dlabel = data["int_label"]
dlabel = dlabel[:qnt]

dsecao = data["secao"]
dsecao = dsecao[:qnt]

dtext



0      LEI Nº 17.597 /2009  EMENTA: DISPÕE SOBRE A SE...
1      DECRETO Nº 25.024 DE 30 DE DEZEMBRODE 2009  EM...
2      PORTARIA Nº 3130 DE 30 DE DEZEMBRO DE  2009  O...
3      º 18012078.    PORTARIA Nº 3137 DE 30 DE DEZEM...
4      esenvolvimento Urbano e Ambiental, a contar da...
                             ...                        
995    a oitenta e sete por cento), do valor global d...
996    DESPACHOS EXARADOS P/ GABINETE DO SECRETARIO  ...
997    FUNDAÇÃO DE CULTURA CIDADE DO RECIFE  GERÊNCIA...
998    FUNDAÇÃO DE CULTURA CIDADE DO RECIFE  Núcleo d...
999    o Recife.    3 - DA APRESENTAÇÃO  3.1.A aprese...
Name: conteudo, Length: 1000, dtype: object

## Clean dataset



In [4]:
def cleanText(text):
   
    text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))  # Remove Punctuations
    text = text.lower()  # Lower case
    text = re.sub(r'[0-9]' , "", text)
    text = re.sub('\s+', ' ', text).strip()  # Remove new line, Tab and double spaces
    text = re.sub(r'\b.{2}\b', ' ', text).strip() # remove terms less than 2 chars in length
    return text





## get corpus

In [5]:
# create corpus

corpus = []
for i in dtext:    
    documentText = cleanText(i)
    corpus.append(documentText)
    
corpus[1]

'decreto       dezembrode ementa abre crédito suplementar  prefeito   cidade   recife   uso das atribuições que lhe confere  inciso     artigo   lei orgânica municipal     abril    tendo   vista  que dispõem  artigo   artigo   inciso iii   artigo   lei       dezembro          art  fica aberto   orçamento   secretaria   educação esporte  lazer  crédito suplementar    trezentos  dez mil cento  vinte  sete reais  oitenta  três centavos destinado   reforço   seguinte dotação orçamentária recursos   tesouro    secretaria   educação esporte  lazer secretaria   educação esporte  lazer administração direta universalização  qualificação   ensino fundamental   material   consumo total art    recursos  serem utilizados para atendimento   que determina  artigo anterior serão anulados das dotações orçamentárias discriminadas  seguir recursos   tesouro    secretaria   educação esporte  lazer secretaria   educação esporte  lazer administração direta incorporação das tecnologias   informação    comuni

## get TF-IDF

In [6]:
ngram = 1
allWords = {}
#get tfidf
    
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, ngram), min_df=2, stop_words=stop_words)
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
tfidf_feature_names_hashmap = {}

# tfidf feature names hashmap
for i in range(0, len(tfidf_feature_names)):
    tfidf_feature_names_hashmap[tfidf_feature_names[i]] = i

#filter based on the mean tf/idf
tfidf_mean = tfidf.mean(0).mean()
words_tfidf = tfidf.mean(0)
for index, item in enumerate(np.nditer(words_tfidf)):
    if item > tfidf_mean:
        allWords[tfidf_feature_names[index]] = 0

allWordsSorted = collections.OrderedDict(sorted(allWords.items()))

## create document document matrix

In [7]:
#create document term matrix (out)
document_term_matrix = ""
for i in range(0, qnt):
    line = ""
    tfidf_hashmap = {}
    for col in tfidf.getrow(i).nonzero()[1]:
        if tfidf_feature_names[col] in allWordsSorted:
            tfidf_hashmap[col] = tfidf[i, col]

    for word, score in allWordsSorted.items():
        word_index = tfidf_feature_names_hashmap.get(word)
        if tfidf_feature_names_hashmap.get(word) in tfidf_hashmap:
            line += str(tfidf_hashmap.get(word_index)) + ","
        else:
            line += "0.0,"
    line = line[0:line.rindex(',')]
    document_term_matrix += line + '\n'

In [8]:
#write document term matrix to file
document_term_matrix_file = open("teste" + ".Matrix", 'w')
document_term_matrix_file.write(document_term_matrix)
document_term_matrix_file.close()

In [9]:
def read_term_document_matrix(path, type="tf"):
    """
    read document-term matrix (comma separate)
    :param path: path to matrix
    :param type: if it is word count of tf/idf value (count or tf)
    :return: 2D matrix (list of list)
    """
    document_term_matrix_file = open(path, 'r')
    matrix = []
    for line in document_term_matrix_file:
        if type == "count":
            line = line.replace('\r','').replace('\n','').replace('0.0','0')
        else:
            line = line.replace('\r', '').replace('\n', '')
        columns = line.split(',')
        row = []
        for column in columns:
            row.append(column)
        matrix.append(row)
    return matrix



#create document-document distance file
document_term_matrix = np.asarray(read_term_document_matrix("teste" + ".Matrix"), dtype=float)
documents_distance = squareform(pdist(document_term_matrix, 'cosine'))
documents_distance_path = "documentDistance"+ str(qnt)
documents_distance_file = open(documents_distance_path, "w")
for i in range(len(document_term_matrix)):
    for j in range(len(document_term_matrix)):
        if j == 0:
            documents_distance_file.write(str(documents_distance[i][j]))
        else:
            documents_distance_file.write("," + str(documents_distance[i][j]))
    documents_distance_file.write("\n")
documents_distance_file.close()

#create labels file
documents_distance_path = "label"+ str(qnt)
documents_distance_file = open(documents_distance_path, "w")
for i in range(len(dlabel)):
    documents_distance_file.write(str(dlabel[i]) + "\n")
documents_distance_file.close()

#create Secao file

documents_distance_path = "secao"+ str(qnt)
documents_distance_file = open(documents_distance_path, "w")
for i in range(len(dsecao)):
    documents_distance_file.write(str(dsecao[i]) + "\n")
documents_distance_file.close()

#create Conteudo file

documents_distance_path = "conteudo"+ str(qnt)
documents_distance_file = open(documents_distance_path, "w")
for i in range(len(dtext)):
    documents_distance_file.write(str(dtext[i]) + "\n")
documents_distance_file.close()

print("DEU CERTO!")

DEU CERTO!
